## 05-1 깔끔한 데이터란? 

- 열은 변수 값의 집합
- 행은 변수 값들의 집합에 대한 각각의 요소(관측값)

## 05-2 열 이름이 값일 때

### `하나의 열만 index 화 하기`

#### [Do It! 실습] 넒은 데이터 확인하기

- 넓은 데이터( wide data )
  - 1개의 변수를 구간으로 나누어 칼럼화
  - 긴 데이터( long data ) 로 변화(unpivot) -> melt, gather

In [ ]:
import pandas as pd

pew = pd.read_csv('../data/pew.csv')

In [ ]:
pew.info()

In [ ]:
print(pew.iloc[:, 0:5])

#### [Do It! 실습] 긴 데이터로 만들기

- melt() 매개변수
  - id_vars : 식별자로 사용할 칼럼 지정
  - value_vars : id_vars 로 지정하지 않은 모든 칼럼이 unpivot 의 대상 지정
  - var_name : unpivot 할 열들을 대표하는 칼럼명 - default = 'variable'
  - value_name : unpivot 후 값을 대표하는 칼럼명 - default = 'value' 

In [ ]:
pew_long = pew.melt(id_vars='religion')
print(pew_long)

In [ ]:
# melt() 메서드
pew_long = pew.melt(id_vars='religion')

# pd.melt() 함수
pew_long = pd.melt(pew, id_vars='religion')

In [ ]:
pew_long = pew.melt(
    id_vars="religion", var_name="income", value_name="count"
)
print(pew_long)

### `여러 개의 열을 index 화 하기`

#### [Do It! 실습] 여러 개 열 유지하기

- 식별자로 사용할 칼럼들을 리스트로 지정

In [ ]:
billboard = pd.read_csv('../data/billboard.csv')
print(billboard.iloc[0:5, 0:16])

In [ ]:
billboard.info()

In [ ]:
billboard_long = billboard.melt(
    id_vars=["year", "artist", "track", "time", "date.entered"],
    var_name="week",
    value_name="rating",
)
print(billboard_long)

- 317 개 -> 24092 개
  - 76 * 317

## 05-3 열 이름에 변수가 여러 개로 분할되어 있을 때

### `열 이름이 여러 가지 뜻일 때`

#### [Do It! 실습] 깔끔한 데이터 만들기 ①

In [ ]:
ebola = pd.read_csv('../data/country_timeseries.csv')
print(ebola.columns)

In [ ]:
ebola.info()

In [ ]:
ebola['Date']=pd.to_datetime(ebola['Date'])

In [ ]:
ebola.info()

In [ ]:
print(ebola.iloc[:5, [0, 1, 2, 10]])

In [ ]:
ebola_long = ebola.melt(id_vars=['Date', 'Day'])
print(ebola_long)

### `열 이름 분할하고 새로운 열로 할당하기`

#### [Do It! 실습] 깔끔한 데이터 만들기 ②

- split 메서드의 output -> 값이 리스트인 시리즈 데이터

In [ ]:
variable_split = ebola_long.variable.str.split('_')
print(variable_split[:5])

In [ ]:
print(type(variable_split))

In [ ]:
print(type(variable_split[0]))

- 리스트 요소별로 지정하여 브로드캐스팅
  - get() 메서드를 사용

In [ ]:
status_values = variable_split.str.get(0)
country_values = variable_split.str.get(1)

In [ ]:
print(status_values)

In [ ]:
type(status_values)

- 한개의 칼럼 값을 2 개의 칼럼 값으로 새열 추가

In [ ]:
ebola_long['status'] = status_values
ebola_long['country'] = country_values
print(ebola_long)

### `한 번에 분할하고 합치기`

#### [Do It! 실습] 깔끔한 데이터 한 번에 만들기

In [ ]:
ebola_long = ebola.melt(id_vars=['Date', 'Day'])

- split 메서드의 output -> 각각의 값으로 나누어진 데이터프레임

In [ ]:
# split the column by _ into a dataframe using expand
variable_split = ebola_long.variable.str.split('_', expand=True)
print(variable_split)

- 구조분해할당(다중 할당 기능 : multiple assignment feature)

In [ ]:
ebola_long[['status', 'country']] = variable_split
print(ebola_long)

- pivot

In [ ]:
ebola_long.pivot_table(
    index=['Date', 'Day', 'variable','country'],
    columns='status',
    values='value'
)

In [ ]:
ebola_long.pivot_table(
    index=['Date', 'Day', 'variable','country'],
    columns='status',
    values='value'
).sort_index(ascending=False)

## 05-4 변수가 행과 열 모두에 있을 때

### `한 열에 변수가 여러개 일때`

- 변수별로 피벗하기

#### [Do It! 실습] 행과 열 모두에 있는 변수 정리하기

In [ ]:
weather = pd.read_csv('../data/weather.csv')
print(weather.iloc[:5, :11])

In [ ]:
weather_melt = weather.melt(
    id_vars=["id", "year", "month", "element"],
    var_name="day",
    value_name="temp",
)

print(weather_melt)

- index : 식별자로 사용할 칼럼(복수 선택 가능)
- columns : pivot 할 칼럼 지정(복수 선택 가능)(칼럼의 값(unique)들이 칼럼명)
- values : pivot 된 칼럼의 값이 될 칼럼 지정

In [ ]:
weather_tidy = weather_melt.pivot_table(
    index=['id', 'year', 'month', 'day'],
    columns='element',
    values='temp'
)
print(weather_tidy)

In [ ]:
weather_tidy_flat = weather_tidy.reset_index()
print(weather_tidy_flat)

In [ ]:
weather_tidy = (
    weather_melt
    .pivot_table(
        index=['id', 'year', 'month', 'day'],
        columns='element',
        values='temp')
    .reset_index()
)
print(weather_tidy)

In [10]:
import pandas as pd

In [11]:
df = pd.read_excel("C:/Users/asiae/Downloads/inventory_Seoul.xlsx")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   품목명      88 non-null     object
 1   로케이션명    88 non-null     object
 2   재고수량(A)  88 non-null     int64 
 3   품목코드     88 non-null     object
 4   창고코드     88 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 3.6+ KB
